In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        (os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# ============================================================
# Weakly Supervised MRI Segmentation using UNETR (MONAI)
# Author: Your Name
# Journal-ready implementation
# ============================================================

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from monai.networks.nets import UNETR
from monai.losses import DiceLoss
from monai.transforms import Compose, Resize, ScaleIntensity

from skimage.filters import threshold_otsu
from scipy.spatial.distance import directed_hausdorff

# ---------------- CONFIG ----------------
IMG_SIZE = 224
BATCH_SIZE = 4
EPOCHS = 20
LR = 1e-4

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
IMAGE_DIR = "/kaggle/input/brain-tumor-for-segmentation/Brain tumor segmentation"
MODEL_PATH = "unetr_weakly_supervised_mri.pth"


# ---------------- PSEUDO MASK GENERATION ----------------
def generate_pseudo_mask(img):
    thresh = threshold_otsu(img)
    mask = (img > thresh).astype(np.uint8)

    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    return mask


# ---------------- DATASET ----------------
class MRIPseudoDataset(Dataset):
    def __init__(self, image_dir):
        self.image_dir = image_dir
        self.images = sorted(os.listdir(image_dir))

        self.transforms = Compose([
            Resize((IMG_SIZE, IMG_SIZE)),
            ScaleIntensity()
        ])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

        mask = generate_pseudo_mask(img)

        img = img.astype(np.float32) / 255.0
        mask = mask.astype(np.float32)

        img = torch.tensor(img).unsqueeze(0)
        mask = torch.tensor(mask).unsqueeze(0)

        img = self.transforms(img)

        return img, mask


# ---------------- METRICS ----------------
def dice_score(pred, gt, eps=1e-7):
    pred = pred.flatten()
    gt = gt.flatten()
    intersection = (pred * gt).sum()
    return (2 * intersection + eps) / (pred.sum() + gt.sum() + eps)


def iou_score(pred, gt, eps=1e-7):
    pred = pred.flatten()
    gt = gt.flatten()
    intersection = (pred * gt).sum()
    union = pred.sum() + gt.sum() - intersection
    return (intersection + eps) / (union + eps)


def hausdorff_distance(pred, gt):
    pred_pts = np.argwhere(pred > 0)
    gt_pts = np.argwhere(gt > 0)
    if len(pred_pts) == 0 or len(gt_pts) == 0:
        return 0.0
    return max(
        directed_hausdorff(pred_pts, gt_pts)[0],
        directed_hausdorff(gt_pts, pred_pts)[0]
    )


# ---------------- DATALOADER ----------------
dataset = MRIPseudoDataset(IMAGE_DIR)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)


# ---------------- MODEL ----------------
model = UNETR(
    img_size=(IMG_SIZE, IMG_SIZE),
    in_channels=1,
    out_channels=1,
    feature_size=16,
    hidden_size=768,
    mlp_dim=3072,
    num_heads=12,
    pos_embed="perceptron",
    norm_name="instance",
    res_block=True,
).to(DEVICE)

loss_fn = DiceLoss(sigmoid=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)


# ---------------- TRAINING ----------------
model.train()
for epoch in range(EPOCHS):
    epoch_loss = 0
    for imgs, masks in loader:
        imgs, masks = imgs.to(DEVICE), masks.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = loss_fn(outputs, masks)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{EPOCHS}] | Loss: {epoch_loss/len(loader):.4f}")


# ---------------- EVALUATION ----------------
model.eval()
dice_list, iou_list, hd_list = [], [], []

with torch.no_grad():
    for imgs, masks in loader:
        imgs = imgs.to(DEVICE)
        outputs = torch.sigmoid(model(imgs)) > 0.5

        outputs = outputs.cpu().numpy()
        masks = masks.numpy()

        for i in range(outputs.shape[0]):
            dice_list.append(dice_score(outputs[i,0], masks[i,0]))
            iou_list.append(iou_score(outputs[i,0], masks[i,0]))
            hd_list.append(hausdorff_distance(outputs[i,0], masks[i,0]))

print("Dice Score:", np.mean(dice_list))
print("IoU Score:", np.mean(iou_list))
print("Hausdorff Distance:", np.mean(hd_list))


# ---------------- SAVE MODEL ----------------
torch.save(model.state_dict(), MODEL_PATH)
print("Model saved successfully.")


ModuleNotFoundError: No module named 'monai'